# Generate data using Hookean dumbbell model

The script will first draw a distribution of molecules and equilibrate. Then for each scenario, simulate, and save in individual files.


In [13]:
import numpy as np
from dilutebrowniandynamics.simulate import simulate_batch
from dilutebrowniandynamics.molecules.Hookean_dumbbell import HookeanDumbbell

from scipy.interpolate import interp1d

from ipywidgets import interactive, fixed
import ipywidgets as widgets

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 8]

## Parameters definition

In [20]:
n_ensemble = 100    # Number of molecules to simulate
dt = .001           # Time step value (dimensionless)
n_proc = 16         # Number of processor cores to use
                    
# Name: file
scenarios = {
    'turbulence_0': 'inputs/turbulence/scenario_turbulence_0.npz',
#    'inkjet_0': 'inputs/inkjet/scenario_inkjet_0.npz',
    'contraction_0': 'inputs/contraction/scenario_contraction_0.npz',
    'elongation_0': 'inputs/elongation/scenario_elongation_0.npz',
    'random_0': 'inputs/random/scenario_random_0.npz'
}

## Initialise molecules
Here we just draw random vectors from a normal distribution with variance ⅓ in each coordinate so that the average norm is 1. To do this with call `from_normal_distribution` constructor. Note that we pass a random seed (`SeedSequence` object) to ensure thread-safe random number generation and reproducible results.

In [4]:
seq = np.random.SeedSequence(2022)
seeds = seq.spawn(n_ensemble)
# Starting vectors
molecules = [HookeanDumbbell.from_normal_distribution(seed)
             for seed in seeds]

## Initial relaxation
Molecular dynamics simulations are usually preceeded by a relaxation/thermalisation step, which in our case is just a simulation with no velocity gradient. Although we know that for Hookean dumbbell the relaxed distribution is the one we just used to create new molecules, we still do it as an example of a first call to the `simulate_batch` function.

In [6]:
gradU = np.zeros((3, 3))
n_rec = int(3/dt)


observables, molecules_init = simulate_batch(molecules, gradU, n_rec, dt, n_proc)

Physical time to compute: 3.0
Calculation started on 16 cores.


100%|██████████| 100/100 [00:01<00:00, 58.65it/s]


We can investigate the dumbbell distribution in a number of ways. Let's look at time series and histograms.

## Simulations

Now let's simulate each scenario. We will use the interpolator `scipy.interpolate.interp1d` with the property `kind=previous` for the sudden elongation and random tensor, `kind=linear` for realistic flows.

In [21]:
simulation_results = []

for name, file in scenarios.items():

    # Build scenario
    # --------------
    scenario = np.load(file)
    t_base = scenario['t']
    t_base = t_base - t_base[0]
    gradU_base = scenario['gradU']
    if name.startswith('elongation') or name.startswith('random'):
        kind='previous'
    else:
        kind='linear'
    gradU = interp1d(t_base, gradU_base, axis=0, kind=kind, bounds_error=False, 
                     fill_value=(gradU_base[0], gradU_base[-1]), assume_sorted=True)
    
    # Simulate
    # --------
    print(f"Simulate {name}...")
    
    n_rec = int(t_base[-1]/dt)
    observables, _ = simulate_batch(molecules_init, gradU, n_rec, dt, n_proc)
    observables['t'] = np.arange(n_rec)*dt
    observables['gradU'] = gradU(np.arange(n_rec)*dt)
    simulation_results.append(observables)

Simulate turbulence_0...
Physical time to compute: 89.898
Calculation started on 16 cores.


100%|██████████| 100/100 [01:38<00:00,  1.02it/s]


Simulate contraction_0...
Physical time to compute: 38.917
Calculation started on 16 cores.


100%|██████████| 100/100 [00:41<00:00,  2.39it/s]


Simulate elongation_0...
Physical time to compute: 6.0
Calculation started on 16 cores.


100%|██████████| 100/100 [00:06<00:00, 14.96it/s]


Simulate random_0...
Physical time to compute: 6.0
Calculation started on 16 cores.


100%|██████████| 100/100 [00:06<00:00, 14.43it/s]


## Inspect

In [23]:
def inspect(scenario, line):
    figure, ax = plt.subplots(nrows=3)
    ax[0].plot(simulation_results[scenario]['A_average'][:,line,0], c='#0c2c84', linewidth=2, label='ix')
    ax[0].plot(simulation_results[scenario]['A_average'][:,line,1], c='#1d91c0', linewidth=2, label='iy')
    ax[0].plot(simulation_results[scenario]['A_average'][:,line,2], c='#7fcdbb', linewidth=2, label='iz')
    ax[0].set_title('Conformation tensor')
    ax[0].legend()

    ax[1].plot(simulation_results[scenario]['S_average'][:,line,0], c='#0c2c84', linewidth=2, label='ix')
    ax[1].plot(simulation_results[scenario]['S_average'][:,line,1], c='#1d91c0', linewidth=2, label='iy')
    ax[1].plot(simulation_results[scenario]['S_average'][:,line,2], c='#7fcdbb', linewidth=2, label='iz')
    ax[1].set_title('Stress tensor')
    
    ax[2].plot(simulation_results[scenario]['gradU'][:,line,0], c='#0c2c84', linewidth=2, label='ix')
    ax[2].plot(simulation_results[scenario]['gradU'][:,line,1], c='#1d91c0', linewidth=2, label='iy')
    ax[2].plot(simulation_results[scenario]['gradU'][:,line,2], c='#7fcdbb', linewidth=2, label='iz')
    ax[2].set_title('Grad(U) tensor')
    
    plt.show()


w = interactive(inspect, scenario=widgets.Dropdown(
                    options=list(zip(scenarios.keys(), range(len(scenarios)))),
                    value=0,
                    description='Scenario:',
                    disabled=False,
                    ),
                    line=widgets.Dropdown(
                    options=[('xx, xy, xz', 0),('yx, yy, yz', 1),('zx, zy, zz', 2)],
                    value=0,
                    description='Line:',
                    disabled=False,
                    )
                
               )
w

interactive(children=(Dropdown(description='Scenario:', options=(('turbulence_0', 0), ('contraction_0', 1), ('…

## Save result to files

Here we save the result as *numpy arrays* compressed in a single file, as we expect further analysis with Python. Use other tools to save as text files.

In [24]:
params = {'model' : molecules[0].__class__.__name__,
          'model_attributes' : molecules[0].__dict__,
          'n_ensemble' : n_ensemble,
          'dt' : dt,
          'n_proc' : n_proc
          }

for i, name in enumerate(scenarios.keys()):
    observables = simulation_results[i]
    np.savez_compressed(f"outputs/Hookean/{name}", **observables, params=params)